# Installing and Importing Required Libraries

In [1]:
#Installing the required Libraries

!pip install -U -q tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Importing required libraries

from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb


In [4]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Reading, Processing and Chunking the dataset.

In [300]:
# The myntra dataset
df = pd.read_csv('https://raw.githubusercontent.com/Unnathi-Gowda54/RELvouge/main/fashion_dataset.csv')

In [6]:
df.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [7]:
# Checking size of the dataset
df.shape

(14214, 11)

In [8]:
# Checking for missing values
df.isnull().sum()

p_id               0
name               0
products           0
price              0
colour             0
brand              0
img                0
ratingCount     7684
avg_rating      7684
description        0
p_attributes       0
dtype: int64

In [9]:
# There seems to be a lot of missing values in the columns "ratingCount" and "avg_rating"
# imputing the missing values
df['ratingCount'].fillna(df['ratingCount'].mean(), inplace=True)
df['avg_rating'].fillna(df['avg_rating'].mean(), inplace=True)

###### Preparing the data by lowering texts, removing html tags etc.,


In [10]:
df['name'] = df['name'].apply(lambda x: x.lower())

In [11]:
df['products'] = df['products'].apply(lambda x: x.lower())

In [12]:
df['colour'].value_counts()

colour
Black                1901
Blue                 1803
Pink                 1170
Green                1082
Navy Blue             842
White                 807
Red                   707
Grey                  571
Maroon                521
Yellow                496
Beige                 486
Mustard               325
Off White             313
Peach                 298
Purple                294
Orange                287
Olive                 252
Brown                 250
Teal                  192
Cream                 156
Multi                 147
Burgundy              144
Sea Green             115
Turquoise Blue        113
Rust                  108
Lavender               99
Gold                   91
Magenta                86
Charcoal               84
Coral                  81
Fuchsia                52
Mauve                  51
Lime Green             50
Rose                   44
Grey Melange           43
Khaki                  27
Coffee Brown           25
Taupe                  18
Fluor

In [13]:
# Preprocessing Texts
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # Removing HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Removing punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Removing extra spaces
    return text

df['description'] = df['description'].apply(preprocess_text)
df['p_attributes'] = df['p_attributes'].apply(preprocess_text)


In [14]:
df.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,khushal k women black ethnic motifs printed ku...,"kurta, palazzos, dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,black printed kurta with palazzos with dupatta...,addons na body shape id 443333324424 body or g...
1,16524740,inweave women orange solid kurta with palazzos...,"kurta, palazzos, floral print dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,orange solid kurta with palazzos with dupattak...,addons na body shape id 443333324424 body or g...
2,16331376,anubhutee women navy blue ethnic motifs embroi...,"kurta, trousers, dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,navy blue embroidered kurta with trousers with...,addons na body shape id 333424 body or garment...
3,14709966,nayo women red floral printed kurta with trous...,"kurta, trouser, dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,red printed kurta with trouser and dupattakurt...,addons na body shape id 333424 body or garment...
4,11056154,ahika women black & green printed straight kurta,kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,black and green printed straight kurta has a n...,body shape id 424 body or garment size garment...


In [15]:
# Converting the data type to string
df['price'] = df['price'].astype(str)
df['ratingCount'] = df['ratingCount'].astype(str)
df['avg_rating'] = df['avg_rating'].astype(str)

In [173]:
# A new column to store Product Information
df['Product_Info'] = 'name: '+df['name']+'.  '+ 'products: '+ df['products']+'.  '+'price: '+df['price']+'.  '+'colour: '+ df['colour']+'.  '+ 'brand: '+ df['brand']+'.  "'+ 'image_url: '+ df['img']+'".  '+ 'rating count: '+ df['ratingCount']+'.  '+ 'average rating: '+ df['avg_rating']+'.  '+ 'description: '+ df['description']+'.  '+ 'Product Details: '+ df['p_attributes']+"."

In [161]:
# A column to store MetaData
df['Metadata'] = df.apply(lambda x: {'Name': x['name'], 'Product_ID': x['p_id'], "Price": x['price']}, axis=1)

In [163]:
# A column for length of the text.
df['text_len'] = df['Product_Info'].apply(lambda x: len(x.split(" ")))

In [164]:
# Checking if there are any product information with very less data
df['text_len'].describe() # minimum length is 32 and maximum length is 372

count    14214.000000
mean       127.195089
std         38.237430
min         32.000000
25%        106.000000
50%        124.000000
75%        144.000000
max        372.000000
Name: text_len, dtype: float64

In [174]:
df.head(1)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes,Product_Info,Metadata,text_len
0,17048614,khushal k women black ethnic motifs printed ku...,"kurta, palazzos, dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.4183989385227775,black printed kurta with palazzos with dupatta...,addons na body shape id 443333324424 body or g...,name: khushal k women black ethnic motifs prin...,{'Name': 'khushal k women black ethnic motifs ...,203


# Generating and Storing Embeddings using OpenAI and ChromaDB

In [25]:
# Set the API key

filepath = "/content/drive/MyDrive/"
with open(filepath + "OpenAI_API_Key.txt","r") as f:
  openai.api_key = ' '.join(f.readlines())


In [26]:
# Importing the OpenAI Embedding Function into Chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [27]:
# Defining a path where the chroma collections will be stored

chroma_data_path = '/content/drive/MyDrive/Upgrad_Gen_AI/RELvouge/ChromaDB_Data'

In [28]:
# importing cromadb
import chromadb

In [29]:
# Calling PersistentClient()

client = chromadb.PersistentClient()

In [30]:
# Setting up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name = model)

In [176]:
# Initialising a collection in chroma and passing the embeddings to it

Fashion_collection = client.get_or_create_collection(name = 'RAG_Fashion', embedding_function = embedding_function)

In [177]:
# Converting the data into lists

product_details_list = df['Product_Info'].tolist()
meta_data_list = df['Metadata'].tolist()


In [178]:
# Adding documents, metadata to the chromadb collections
import time

# Function to add documents with a delay between batches
def add_documents_with_delay(documents_list, ids_list, metadatas_list, batch_size, delay_between_batches):
    num_batches = len(documents_list) // batch_size + (1 if len(documents_list) % batch_size != 0 else 0)

    for batch_index in range(num_batches):
        start_index = batch_index * batch_size
        end_index = min((batch_index + 1) * batch_size, len(documents_list))

        batch_documents = documents_list[start_index:end_index]
        batch_ids = ids_list[start_index:end_index]
        batch_metadatas = metadatas_list[start_index:end_index]

        Fashion_collection.add(
            documents=batch_documents,
            ids=batch_ids,
            metadatas=batch_metadatas
        )

        if batch_index < num_batches - 1:
            time.sleep(delay_between_batches)  # Delay between batches

# Example usage:
add_documents_with_delay(product_details_list, [str(i) for i in range(len(product_details_list))], meta_data_list, batch_size=1000, delay_between_batches=1)


In [180]:
check = Fashion_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings','documents','metadatas']
)

In [181]:
print(check['documents'])

['name: khushal k women black ethnic motifs printed kurta with palazzos & with dupatta.  products: kurta, palazzos, dupatta.  price: 5099.0.  colour: Black.  brand: Khushal K.  "image_url: http://assets.myntassets.com/assets/images/17048614/2022/2/4/b0eb9426-adf2-4802-a6b3-5dbacbc5f2511643971561167KhushalKWomenBlackEthnicMotifsAngrakhaBeadsandStonesKurtawit7.jpg".  rating count: 4522.0.  average rating: 4.4183989385227775.  description: black printed kurta with palazzos with dupatta kurta design ethnic motifs printed anarkali shape regular style mandarin collar threequarter regular sleeves calf length with flared hem viscose rayon machine weave fabric palazzos design printed palazzos elasticated waistband slipon closure dupatta length 243 meters widthnbsp88 cmthe model height 58 is wearing a size s100 rayonmachine wash.  Product Details: addons na body shape id 443333324424 body or garment size garment measurements in bottom closure slipon bottom fabric viscose rayon bottom pattern pri

# Creating Cache


In [182]:
# Cache collection to store the queries to reduce further unnecessary search in the main collection
cache_collection = client.get_or_create_collection(name = 'RELvouge_cache', embedding_function = embedding_function)

In [183]:
cache_collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

# Semantic Search with Cache

In [288]:
def semantic_search_with_cache(query, cache_collection=cache_collection, fashion_collection= Fashion_collection, threshold=0.2):
    # Query the cache collection
    cache_results = cache_collection.query(
        query_texts=query,
        n_results=1
    )

    ids = []
    documents = []
    distances = []
    metadatas = []
    results_df = pd.DataFrame()

    # If cache is empty or distance is above threshold, query main collection
    if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
        # Query the main collection for results
        results = fashion_collection.query(
            query_texts=query,
            n_results=10
        )

        # Prepare keys and values for caching
        Keys = []
        Values = []
        for key, val in results.items():
            if val is None:
                continue
            for i in range(min(10, len(val[0]))):
                Keys.append(f"{key}{i}")
                Values.append(str(val[0][i]))

        # Add the query and results to the cache collection
        cache_collection.add(
            documents=[query],
            ids=[query],
            metadatas=dict(zip(Keys, Values))
        )

        #print("Not found in cache. Found in main collection.")

        # Create DataFrame from results
        result_dict = {
            'Metadatas': results['metadatas'][0],
            'Documents': results['documents'][0],
            'Distances': results['distances'][0],
            'IDs': results['ids'][0]
        }
        results_df = pd.DataFrame.from_dict(result_dict)

    # If found in cache, retrieve results from cache
    elif cache_results['distances'][0][0] <= threshold:
        cache_result_dict = cache_results['metadatas'][0][0]

        # Parse cache results
        for key, value in cache_result_dict.items():
            if 'ids' in key:
                ids.append(value)
            elif 'documents' in key:
                documents.append(value)
            elif 'distances' in key:
                distances.append(value)
            elif 'metadatas' in key:
                metadatas.append(value)

        #print("Found in cache!")

        # Create DataFrame from cached results
        results_df = pd.DataFrame({
            'IDs': ids,
            'Documents': documents,
            'Distances': distances,
            'Metadatas': metadatas
        })

    return results_df

# Reranking with Cross-Encoders

In [191]:
# Importing cross encoder
from sentence_transformers import CrossEncoder, util


In [126]:
# initialising the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-macro-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [283]:
# Re-ranking the results from the semantic search using cross encoders
def rerank_with_cross_encoder(query, results_df, cross_encoder=cross_encoder):
  cross_inputs = [[query,response] for response in results_df['Documents']]
  cross_outputs = cross_encoder.predict(cross_inputs)
  results_df['Reranked_scores'] = cross_outputs
  return results_df

# Retrieval Augmented Generation

In [292]:
# A function to generate the response.

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are an expert fashion assistant specializing in detailed product information. Your goal is to provide clear and helpful responses based on the user's query."},
                {"role": "user", "content": f"""You are a helpful assistant in the Fashion domain who can effectively answer user queries about Clothes and Fashion Products.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of products dataset in the dataframe '{results_df}'. These search results are essentially complete detail that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual information about the cloth or the product and the column 'metadata' contains the Product Identification details.

                                                Use the documents in '{results_df}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant product name and detail.

                                                Follow the guidelines below when performing the task.
                                                0. Provide the details in 'documents' section in the user-friendly interactive format like a response.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has urls with relevant information, please reformat the url and return the final information.
                                                3. Use the Metadatas columns in the dataframe to retrieve and mention the product name(s).
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific products.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.
                                                6. Don't mention the internal structure of the dataframe.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the product identification details.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

# Integration of all layers

In [296]:
def Assisstant():
  query = input("User : ")
  if query != "exit":
    results_df = semantic_search_with_cache(query)
    reranked_results_df = rerank_with_cross_encoder(query,results_df)
    top_3 = reranked_results_df.sort_values(by='Reranked_scores', ascending = False).head(3)
    print("\nTop Three Results from Search Layer\n")
    display(top_3)
    Top_3_RAG = top_3[['Documents','Metadatas']]
    response = generate_response(query, reranked_results_df)
    print("\nResult from the Generation Layer\n")
    print("assisstant : ", "\n".join(response))

In [298]:
Assisstant()

User : I need a Kurta with palazzo and duppatta from Libas brand

Top Three Results from Search Layer



,IDs,Documents,Distances,Metadatas,Reranked_scores
0,420,name: libas women black floral printed gotta p...,0.2508014440536499,{'Name': 'libas women black floral printed got...,6.528973
7,27,name: libas women maroon printed kurta with pa...,0.2626999318599701,{'Name': 'libas women maroon printed kurta wit...,6.411516
6,418,name: libas women pink printed straight kurta ...,0.26218146085739136,{'Name': 'libas women pink printed straight ku...,6.378007



Result from the Generation Layer

assisstant :  I found a Libas product that matches your search query for a Kurta with palazzo and dupatta:

- Name: Libas Women Maroon Printed Kurta with Palazzo and Dupatta
- Details: This set includes a maroon printed kurta paired with matching palazzo pants and a dupatta.
- Product ID: 27
- Distance: 0.2626999318599701

This product should meet your requirements. If you need further assistance or have more specific preferences, feel free to ask!


In [299]:
Assisstant()

User : I'm looking for a kurta with trousers

Top Three Results from Search Layer



,IDs,Documents,Distances,Metadatas,Reranked_scores
5,484,name: sangria women pink & white printed kurta...,0.2845028042793274,{'Name': 'sangria women pink & white printed k...,-0.563576
0,380,name: here&now women blue ethnic motifs printe...,0.2668187916278839,{'Name': 'here&now women blue ethnic motifs pr...,-2.665724
8,517,name: aasiya women yellow floral printed regul...,0.2894784212112427,{'Name': 'aasiya women yellow floral printed r...,-2.872107



Result from the Generation Layer

assisstant :  I found a couple of kurta options that come with trousers for you:

1. Product Name: here&now Women Blue Ethnic Motifs Printed Kurta with Trousers
2. Product Name: gvs shoppe Women Maroon Embroidered Kurta with Trousers

These options should meet your requirement for a kurta with trousers. Let me know if you would like more details on these products or if you're interested in other options.


In [297]:
Assisstant()

User : I want a kurta which is costs lesser than 3000 rupees.

Top Three Results from Search Layer



,IDs,Documents,Distances,Metadatas,Reranked_scores
1,7607,name: fabindia women beige striped cotton kurt...,0.314365416765213,{'Name': 'fabindia women beige striped cotton ...,-4.529969
2,7969,name: indya blue striped embroidered kurta. p...,0.3188077211380005,{'Name': 'indya blue striped embroidered kurta...,-4.774655
7,627,name: fabindia women white & pink pure cotton ...,0.324074923992157,{'Name': 'fabindia women white & pink pure cot...,-5.036171



Result from the Generation Layer

assisstant :  Certainly! Based on your query for a kurta priced below 3000 rupees, here is a relevant option from the available dataset:

**Product Name:** Fabindia Women Pink Ethnic Motifs Printed Kurta  
**Price:** Below 3000 rupees  

For more details and to explore additional options, feel free to browse through the provided dataset.
